# Generate ALOGIT input from ActivitySim Estimation Data Bundles

This notebook reads tour destination choice estimation data bundles and generates ALOGIT format inputs for destination choice estimation  


## Load Libraries

In [1]:
import pandas as pd
pd.set_option("display.max_columns",250)
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
import openmatrix as omx
from datetime import datetime

SKIM_PATH = r'E:\Met_Council\metc-asim-model\Base_2018\OMX\allskims.omx'
SKIM_TABLE = 'DIST'

In [2]:
omx_file = omx.open_file(SKIM_PATH, 'r')
#dist_skim = np.zeros([])

dist_skim = np.array(omx_file[SKIM_TABLE])
omx_file.close()

## Load Estimation Data Bundle

In [3]:
asim_output_folder = r'E:\Met_Council\metc-asim-model\estimation\output'
asim_data_folder = r'E:\Met_Council\metc-asim-model\estimation\data'
edb_folder = os.path.join(asim_output_folder, 'estimation_data_bundle')
mwcog_data_folder = r'E:\Met_Council\survey_data'
output_folder = r'E:\Met_Council\metc-asim-model\estimation\output\work_loc_estimation'

In [4]:
work_location_folder = os.path.join(edb_folder, 'workplace_location')

work_chooser_create_time = os.path.getctime(os.path.join(work_location_folder, 'workplace_location_choosers_combined.csv'))
work_alts_create_time = os.path.getctime(os.path.join(work_location_folder, 'workplace_location_alternatives_combined.csv'))
work_landuse_create_time = os.path.getctime(os.path.join(work_location_folder, 'workplace_location_landuse.csv'))

print(f"Chooser file timestamp: {datetime.fromtimestamp(work_chooser_create_time).strftime('%c')}")
print(f"Alternative file timestamp: {datetime.fromtimestamp(work_alts_create_time).strftime('%c')}")
print(f"Land use file timestamp: {datetime.fromtimestamp(work_landuse_create_time).strftime('%c')}")

Chooser file timestamp: Fri Jan 19 16:17:41 2024
Alternative file timestamp: Fri Jan 19 16:22:19 2024
Land use file timestamp: Fri Jan 19 14:05:28 2024


## Converting ActivitySim Estimation Output to ALOGIT input

ALOGIT tables are of the form:

person_id, taz1_dist, taz2_dist,.... taz1_logsum, taz2_logsum, .... person_vars, hh_vars

and taz characteristics file:

taz cols: taz1, taz2, taz3

rows: size term segment1

      size term segment2
      
      ...

In [5]:
asim_per_df = pd.read_csv(os.path.join(asim_output_folder, 'final_persons.csv'))
asim_hh_df = pd.read_csv(os.path.join(asim_output_folder, 'final_households.csv'))

override_per_df_in = pd.read_csv(os.path.join(asim_data_folder, 'override_persons.csv'))
override_hh_df_in = pd.read_csv(os.path.join(asim_data_folder, 'override_households.csv'))

In [6]:
# overrides have multiple days of data, move down to one

override_hh_df = override_hh_df_in.groupby('HH_ID').first().reset_index()
override_hh_df

,HH_ID,household_id,home_zone_id,income,hhsize,HHT,auto_ownership,num_workers,children,day,transponder_ownership,survey_year,day.1,chauf_id1,chauf_id2,child_id1,child_id2,child_id3,school_escorting_outbound,school_escorting_inbound,school_escorting_outbound_cond,joint_tour_frequency,joint_tour_frequency_composition,has_joint_tour
0,18112062,14033,756,76750.73,5,1,3,1,3,4,True,2016,4,NaN,NaN,NaN,NaN,NaN,1,1,1,0_tours,0,0
1,18114244,1,1191,167.24,6,1,0,0,4,1,False,2016,1,NaN,NaN,NaN,NaN,NaN,1,1,1,0_tours,0,0
2,18114304,2,2931,103374.66,1,6,1,1,0,1,True,2016,1,NaN,NaN,NaN,NaN,NaN,1,1,1,0_tours,0,0
3,18114536,3,2583,81633.46,2,1,2,0,0,1,True,2016,1,NaN,NaN,NaN,NaN,NaN,1,1,1,0_tours,0,0
4,18114695,4,2651,85278.84,4,1,2,2,2,1,True,2016,1,NaN,NaN,NaN,NaN,NaN,1,1,1,1_Shop,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15663,22002786,27090,1130,113482.00,4,1,2,2,2,1,True,2022,1,63595.0,63594.0,63596.0,63597.0,NaN,1,1,1,0_tours,0,0
15664,22002788,27091,1931,140538.00,3,1,2,2,1,1,True,2022,1,79998.0,79999.0,80000.0,NaN,NaN,1,1,1,0_tours,0,0
15665,22002789,27092,1107,116237.00,3,1,2,2,1,1,True,2022,1,80002.0,80001.0,80003.0,NaN,NaN,1,1,1,0_tours,0,0
15666,22002790,27093,2048,80743.00,3,1,2,1,1,1,True,2022,1,NaN,NaN,NaN,NaN,NaN,1,1,1,0_tours,0,0


In [7]:
override_per_df = override_per_df_in.groupby(['HH_ID', 'PER_ID']).first().reset_index()
override_per_df

,HH_ID,PER_ID,person_id,household_id,PNUM,age,SEX,pemploy,pstudent,is_student,ptype,school_zone_id,workplace_zone_id,free_parking_at_work,work_from_home,telecommute_frequency,day,educ,transit_pass_subsidy,transit_pass_ownership,cdap_activity,mandatory_tour_frequency,_escort,_shopping,_othmaint,_othdiscr,_eatout,_social,non_mandatory_tour_frequency
0,18112062,1,29039,14033,1,50,1,1,3,False,1,-1.0,2812.0,True,False,No_Telecommute,4,13,0,0,M,work1,0,0,0,0,0,0,0
1,18112062,2,29040,14033,2,45,2,3,3,False,4,-1.0,-1.0,False,False,No_Telecommute,4,13,0,0,H,None,0,0,0,0,0,0,0
2,18112062,3,29041,14033,3,6,2,4,1,True,7,-1.0,-1.0,False,False,No_Telecommute,4,0,0,0,H,None,0,0,0,0,0,0,0
3,18112062,4,29042,14033,4,2,2,4,1,True,8,-1.0,-1.0,False,False,No_Telecommute,4,0,0,0,H,None,0,0,0,0,0,0,0
4,18112062,5,29043,14033,5,2,2,4,1,True,8,-1.0,-1.0,False,False,No_Telecommute,4,0,0,0,H,None,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31623,22002790,1,55275,27093,1,33,2,3,3,False,4,-1.0,-1.0,False,False,No_Telecommute,1,13,0,0,H,None,0,0,0,0,0,0,0
31624,22002790,2,55276,27093,2,1,2,4,1,True,8,-1.0,-1.0,False,False,No_Telecommute,1,0,0,0,H,None,0,0,0,0,0,0,0
31625,22002790,3,55277,27093,3,32,1,1,3,False,1,-1.0,2073.0,False,False,No_Telecommute,1,13,0,0,H,work1,0,0,0,0,0,0,0
31626,22002791,1,55278,27094,1,53,2,1,3,False,1,-1.0,1111.0,False,False,No_Telecommute,1,13,0,0,H,work1,0,0,0,0,0,0,0


In [8]:
override_per_df.ptype.value_counts().sort_index()

1    14499
2     2218
3      809
4     1680
5     6779
6      458
7     3463
8     1722
Name: ptype, dtype: int64

In [9]:
pd.crosstab(override_per_df.ptype, override_per_df.workplace_zone_id >= 0, margins = True).style.format("{:,.0f}")

workplace_zone_id,False,True,All
ptype,,,
1,"5,642","8,857","14,499"
2,"1,007","1,211","2,218"
3,631,178,809
4,"1,673",7,"1,680"
5,"6,746",33,"6,779"
6,392,66,458
7,"3,463",0,"3,463"
8,"1,722",0,"1,722"
All,"21,276","10,352","31,628"


In [10]:
pd.DataFrame(override_per_df_in.ptype.value_counts()).join(pd.DataFrame(override_per_df.ptype.value_counts()), lsuffix = '_input', rsuffix = '_deduped').sort_index()

,ptype_input,ptype_deduped
1,39463,14499
2,5504,2218
3,2140,809
4,4193,1680
5,14444,6779
6,1171,458
7,8506,3463
8,4585,1722


In [11]:
asim_per_df['pemploy'].value_counts(dropna=False).sort_index()

1    39498
2     6897
3    20520
4    13091
Name: pemploy, dtype: int64

In [12]:
override_per_df

,HH_ID,PER_ID,person_id,household_id,PNUM,age,SEX,pemploy,pstudent,is_student,ptype,school_zone_id,workplace_zone_id,free_parking_at_work,work_from_home,telecommute_frequency,day,educ,transit_pass_subsidy,transit_pass_ownership,cdap_activity,mandatory_tour_frequency,_escort,_shopping,_othmaint,_othdiscr,_eatout,_social,non_mandatory_tour_frequency
0,18112062,1,29039,14033,1,50,1,1,3,False,1,-1.0,2812.0,True,False,No_Telecommute,4,13,0,0,M,work1,0,0,0,0,0,0,0
1,18112062,2,29040,14033,2,45,2,3,3,False,4,-1.0,-1.0,False,False,No_Telecommute,4,13,0,0,H,None,0,0,0,0,0,0,0
2,18112062,3,29041,14033,3,6,2,4,1,True,7,-1.0,-1.0,False,False,No_Telecommute,4,0,0,0,H,None,0,0,0,0,0,0,0
3,18112062,4,29042,14033,4,2,2,4,1,True,8,-1.0,-1.0,False,False,No_Telecommute,4,0,0,0,H,None,0,0,0,0,0,0,0
4,18112062,5,29043,14033,5,2,2,4,1,True,8,-1.0,-1.0,False,False,No_Telecommute,4,0,0,0,H,None,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31623,22002790,1,55275,27093,1,33,2,3,3,False,4,-1.0,-1.0,False,False,No_Telecommute,1,13,0,0,H,None,0,0,0,0,0,0,0
31624,22002790,2,55276,27093,2,1,2,4,1,True,8,-1.0,-1.0,False,False,No_Telecommute,1,0,0,0,H,None,0,0,0,0,0,0,0
31625,22002790,3,55277,27093,3,32,1,1,3,False,1,-1.0,2073.0,False,False,No_Telecommute,1,13,0,0,H,work1,0,0,0,0,0,0,0
31626,22002791,1,55278,27094,1,53,2,1,3,False,1,-1.0,1111.0,False,False,No_Telecommute,1,13,0,0,H,work1,0,0,0,0,0,0,0


In [13]:
override_per_df[(override_per_df['workplace_zone_id'] >= 0) & (override_per_df['workplace_zone_id'] <= 3030) & (~override_per_df['work_from_home'])].ptype.value_counts().sort_index()

1    8776
2    1203
3     172
4       7
5      32
6      66
Name: ptype, dtype: int64

In [14]:
asim_per_df[(asim_per_df['workplace_zone_id'] >= 0) & (~asim_per_df['work_from_home'])].ptype.value_counts().sort_index()

1    25959
2     3190
3      414
6      138
Name: ptype, dtype: int64

In [15]:
asim_per_df['ptype'].value_counts(dropna=False).sort_index()

1    39463
2     5504
3     2140
4     4193
5    14444
6     1171
7     8506
8     4585
Name: ptype, dtype: int64

## Person and Household Information
ALOGIT doesn't allow for strings in the input files, so the person and household files categorical variables need to be converted to integers

In [16]:
asim_per_hh_df = pd.merge(asim_per_df, asim_hh_df, how='left', on ='household_id')

In [17]:
override_per_df

,HH_ID,PER_ID,person_id,household_id,PNUM,age,SEX,pemploy,pstudent,is_student,ptype,school_zone_id,workplace_zone_id,free_parking_at_work,work_from_home,telecommute_frequency,day,educ,transit_pass_subsidy,transit_pass_ownership,cdap_activity,mandatory_tour_frequency,_escort,_shopping,_othmaint,_othdiscr,_eatout,_social,non_mandatory_tour_frequency
0,18112062,1,29039,14033,1,50,1,1,3,False,1,-1.0,2812.0,True,False,No_Telecommute,4,13,0,0,M,work1,0,0,0,0,0,0,0
1,18112062,2,29040,14033,2,45,2,3,3,False,4,-1.0,-1.0,False,False,No_Telecommute,4,13,0,0,H,None,0,0,0,0,0,0,0
2,18112062,3,29041,14033,3,6,2,4,1,True,7,-1.0,-1.0,False,False,No_Telecommute,4,0,0,0,H,None,0,0,0,0,0,0,0
3,18112062,4,29042,14033,4,2,2,4,1,True,8,-1.0,-1.0,False,False,No_Telecommute,4,0,0,0,H,None,0,0,0,0,0,0,0
4,18112062,5,29043,14033,5,2,2,4,1,True,8,-1.0,-1.0,False,False,No_Telecommute,4,0,0,0,H,None,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31623,22002790,1,55275,27093,1,33,2,3,3,False,4,-1.0,-1.0,False,False,No_Telecommute,1,13,0,0,H,None,0,0,0,0,0,0,0
31624,22002790,2,55276,27093,2,1,2,4,1,True,8,-1.0,-1.0,False,False,No_Telecommute,1,0,0,0,H,None,0,0,0,0,0,0,0
31625,22002790,3,55277,27093,3,32,1,1,3,False,1,-1.0,2073.0,False,False,No_Telecommute,1,13,0,0,H,work1,0,0,0,0,0,0,0
31626,22002791,1,55278,27094,1,53,2,1,3,False,1,-1.0,1111.0,False,False,No_Telecommute,1,13,0,0,H,work1,0,0,0,0,0,0,0


In [18]:
#get work from home variable from the override data
#Asim doesn't report the correct work from home variable
if 'work_from_home' in asim_per_hh_df.columns:
    asim_per_hh_df.drop(columns=['work_from_home'], inplace=True)
if 'SEX' in asim_per_hh_df.columns:
    asim_per_hh_df.drop(columns=['SEX'], inplace=True)

asim_per_hh_df = pd.merge(asim_per_hh_df, override_per_df[['person_id','work_from_home','SEX']], how='left', on ='person_id')

In [19]:
override_hh_df['income'].describe()

count     15668.000000
mean      95085.637609
std       63984.345933
min          15.820000
25%       46890.152500
50%       88021.915000
75%      126165.537500
max      338820.330000
Name: income, dtype: float64

In [20]:
pd.cut(override_hh_df['income'] / 1000.0, bins=[-np.inf, 20, 40, 70, np.inf], labels=[1, 2, 3, 4]).astype(int).value_counts().sort_index()

1    1403
2    1846
3    2919
4    9500
Name: income, dtype: int64

In [21]:
asim_per_hh_df.shape

(80006, 146)

In [22]:
#get income from the override data
# this step will not be needed after the Asim is rerun with fixed income variable
#asim_per_hh_df.drop(columns=['income'], inplace=True)
asim_per_hh_df = pd.merge(asim_per_hh_df, override_hh_df[['household_id','income']], how='left', on ='household_id')

#code income segments
#asim_per_hh_df['income_segment_y'] = 1
#asim_per_hh_df['income_segment_y'] = np.where((asim_per_hh_df['income']>=40000) & (asim_per_hh_df['income']<70000),2,asim_per_hh_df['income_segment_y'])
#asim_per_hh_df['income_segment_y'] = np.where((asim_per_hh_df['income']>=70000) & (asim_per_hh_df['income']<100000),3,asim_per_hh_df['income_segment_y'])
#asim_per_hh_df['income_segment_y'] = np.where((asim_per_hh_df['income']>=100000),4,asim_per_hh_df['income_segment_y'])
asim_per_hh_df['income_segment_y'] = pd.cut(asim_per_hh_df['income'].fillna(0) / 1000.0, bins=[-np.inf, 20, 40, 70, np.inf], labels=[1, 2, 3, 4]).astype(int)

In [23]:
asim_per_hh_df['income_segment_y'].value_counts()

1    50554
4    21545
3     5027
2     2880
Name: income_segment_y, dtype: int64

In [24]:
cdap_activity_dict = {
    'M': 1,
    'N': 2,
    'H': 3
}
asim_per_hh_df['cdap_activity'] = asim_per_hh_df['cdap_activity'].apply(lambda x: cdap_activity_dict[x])

In [25]:
mandatory_tour_freq_dict = {
    'None': 0,
    'work1': 1,
    'school1': 2,
    'work2': 3,
    'school2': 4,
    'work_and_school': 5
}
asim_per_hh_df['mandatory_tour_frequency'] = asim_per_hh_df['mandatory_tour_frequency'].fillna('None').apply(
    lambda x: mandatory_tour_freq_dict[x])

In [26]:
#asim_per_hh_df['joint_tour_frequency'].value_counts().sort_index()

In [27]:
joint_tour_freq_dict = {
    '0_tours': 1,
    '1_Shop': 2,
    '1_Main': 3,
    '1_Eat': 4,
    '1_Visit': 5,
    '1_Disc': 6,
    '2_SS': 7,
    '2_SM': 8,
    '2_SE': 9,
    '2_SV': 10,
    '2_SD': 11,
    '2_MM': 12,
    '2_ME': 13,
    '2_MV': 14,
    '2_MD': 15,
    '2_EE': 16,
    '2_EV': 17,
    '2_ED': 18,
    '2_VV': 19,
    '2_VD': 20,
    '2_DD': 21
}
asim_per_hh_df['joint_tour_frequency'] = asim_per_hh_df['joint_tour_frequency'].apply(lambda x: joint_tour_freq_dict[x])

In [28]:
#asim_per_hh_df['telecommute_frequency'].value_counts().sort_index()

In [29]:
telecommute_frequency_dict = {
    '1_day_week': 1,
    '2_3_days_week': 2,
    '4_days_week': 3,
    'No_Telecommute':4,
    'Not_Applicable':-9
}
asim_per_hh_df['telecommute_frequency'].fillna(value='Not_Applicable', inplace=True)
asim_per_hh_df['telecommute_frequency'] = asim_per_hh_df['telecommute_frequency'].apply(lambda x: telecommute_frequency_dict[x])

In [30]:

metc_counties = {
    'Anoka'     : 1,
    'Carver'    : 2,
    'Chisago'   : 3,
    'Dakota'    : 4,
    'Goodhue'   : 5,
    'Hennepin'  : 6,
    'Isanti'    : 7,
    'Le Sueur'  : 8,
    'McLeod'    : 9,
    'Pierce'    : 10,
    'Polk'      : 11,
    'Ramsey'    : 12,
    'Rice'      : 13,
    'Scott'     : 14,
    'Sherburne' : 15,
    'Sibley'    : 16,
    'St. Croix' : 17,
    'Washington': 18,
    'Wright'    : 19
}

asim_per_hh_df['county'] = asim_per_hh_df['home_county_y'].map(metc_counties)
asim_per_hh_df.drop(columns = ['home_county_y', 'home_county_x'], inplace = True)

In [31]:
asim_per_hh_df['work_from_home'] = asim_per_hh_df['work_from_home'].astype(bool)

for col in asim_per_hh_df.columns:
    if asim_per_hh_df[col].dtype == 'bool':
        asim_per_hh_df[col] = np.where(asim_per_hh_df[col] == True, 1, 0)
        
is_number = np.vectorize(lambda x: np.issubdtype(x, np.number))

for col_name, col_type in zip(asim_per_hh_df.columns, asim_per_hh_df.dtypes):
    if not is_number(col_type):
        print(f"Column {col_name} is not a number (is a {col_type})")

assert all(is_number(asim_per_hh_df.dtypes)), "Have Non-Numeric columns!"

### Tours

In [33]:
# asim_tours_df = pd.read_csv(os.path.join(asim_output_folder, 'final_tours.csv'))

In [34]:
#asim_tours_df.head(-10)

In [35]:
#asim_tours_df.info()

In [36]:
#all(pd.isna(asim_tours_df['destination_logsum']))

In [37]:
#asim_tours_df['tour_type'].value_counts(dropna=False)

In [38]:
tour_type_dict = {
    'work': 1,
    'school': 2,
    'escort': 3,
    'shopping': 4,
    'eatout': 5,
    'othmaint': 6,
    'social': 7,
    'othdiscr': 8,
    'eat': 9,
    'maint': 10,
    'business': 11,
}
#asim_tours_df['tour_type'] = asim_tours_df['tour_type'].apply(lambda x: tour_type_dict[x])

In [39]:
#asim_tours_df['tour_category'].value_counts(dropna=False)

In [40]:
#asim_tours_df['tour_category'].value_counts(dropna=False, normalize=True)

In [41]:
tour_category_dict = {
    'atwork': 4,
    'non_mandatory': 3,
    'joint': 2,
    'mandatory': 1,
}

#asim_tours_df['tour_category'] = asim_tours_df['tour_category'].apply(lambda x: tour_category_dict[x])

In [42]:
#asim_tours_df['composition'].value_counts(dropna=False)

In [43]:
tour_composition_dict = {
    'NA': 0,
    'adults': 1,
    'children': 2,
    'mixed': 3
}
#asim_tours_df['composition'] = asim_tours_df['composition'].fillna('NA').apply(lambda x: tour_composition_dict[x])

In [44]:
#asim_tours_df['tour_mode'].value_counts(dropna=False)

In [46]:
tour_mode_dict = {
    'DRIVEALONE':    1,
    'SHARED2':        2,
    'SHARED3':        3,
    'WALK':           4,
    'SCHOOLBUS':      5,
    'WALK_MR':        6,
    'WALK_AB':         7,
    'PNR_MR':          8,
    'BIKE':            9,
    'WALK_BM':         10,
    'TNC_SINGLE':      11,
    'KNR_MR':          12,
    'TAXI':            13,
    'PNR_CR':          14,
    'PNR_AB':          15,
    'PNR_BM':           16,
    'TNC_SHARED':       17,
    'KNR_BM':           18,
    'WALK_CR':          19,
    'KNR_AB':           20,
    'KNR_CR':           21
}
#asim_tours_df['tour_mode'] = asim_tours_df['tour_mode'].fillna('NA').apply(lambda x: tour_mode_dict[x])

In [47]:
#asim_tours_df['atwork_subtour_frequency'].value_counts(dropna=False)

In [48]:
atwork_subtour_dict = {
    'NA': -1,
    'no_subtours': 0,
    'maint': 1,
    'eat': 2,
    'business1': 3,
    'business2': 4,
    'eat_business': 5,
}

#asim_tours_df['atwork_subtour_frequency'] = asim_tours_df['atwork_subtour_frequency'].fillna("NA").apply(
#    lambda x: atwork_subtour_dict[x])

In [49]:
#asim_tours_per_hh_df = pd.merge(asim_tours_df, asim_per_hh_df, how='left', on=['person_id', 'household_id'])

In [50]:
#assert all(is_number(asim_tours_per_hh_df.dtypes)), "Have Non-Numeric columns!"

In [51]:
#asim_tours_per_hh_df.head()

### Landuse

In [52]:
# read Asim land use output
landuse = pd.read_csv(os.path.join(asim_output_folder, 'final_land_use.csv'))

# filter out external zones
landuse = landuse[landuse['zone_id']<=3030]


landuse_alogit = landuse.pivot_table(columns='zone_id')
landuse_alogit_cols = ['taz' + str(col) for col in landuse_alogit.columns]
landuse_alogit.columns = landuse_alogit_cols
landuse_alogit = landuse_alogit.reset_index().rename(columns={'index': 'landuse_var'})
landuse_alogit

,landuse_var,taz1,taz2,taz3,taz4,taz5,taz6,taz7,taz8,taz9,taz10,taz11,taz12,taz13,taz14,taz15,taz16,taz17,taz18,taz19,taz20,taz21,taz22,taz23,taz24,taz25,taz26,taz27,taz28,taz29,taz30,taz31,taz32,taz33,taz34,taz35,taz36,taz37,taz38,taz39,taz40,taz41,taz42,taz43,taz44,taz45,taz46,taz47,taz48,taz49,taz50,taz51,taz52,taz53,taz54,taz55,taz56,taz57,taz58,taz59,taz60,taz61,taz62,taz63,taz64,taz65,taz66,taz67,taz68,taz69,taz70,taz71,taz72,taz73,taz74,taz75,taz76,taz77,taz78,taz79,taz80,taz81,taz82,taz83,taz84,taz85,taz86,taz87,taz88,taz89,taz90,taz91,taz92,taz93,taz94,taz95,taz96,taz97,taz98,taz99,taz100,taz101,taz102,taz103,taz104,taz105,taz106,taz107,taz108,taz109,taz110,taz111,taz112,taz113,taz114,taz115,taz116,taz117,taz118,taz119,taz120,taz121,taz122,taz123,taz124,...,taz2906,taz2907,taz2908,taz2909,taz2910,taz2911,taz2912,taz2913,taz2914,taz2915,taz2916,taz2917,taz2918,taz2919,taz2920,taz2921,taz2922,taz2923,taz2924,taz2925,taz2926,taz2927,taz2928,taz2929,taz2930,taz2931,taz2932,taz2933,taz2934,taz2935,taz2936,taz2937,taz2938,taz2939,taz2940,taz2941,taz2942,taz2943,taz2944,taz2945,taz2946,taz2947,taz2948,taz2949,taz2950,taz2951,taz2952,taz2953,taz2954,taz2955,taz2956,taz2957,taz2958,taz2959,taz2960,taz2961,taz2962,taz2963,taz2964,taz2965,taz2966,taz2967,taz2968,taz2969,taz2970,taz2971,taz2972,taz2973,taz2974,taz2975,taz2976,taz2977,taz2978,taz2979,taz2980,taz2981,taz2982,taz2983,taz2984,taz2985,taz2986,taz2987,taz2988,taz2989,taz2990,taz2991,taz2992,taz2993,taz2994,taz2995,taz2996,taz2997,taz2998,taz2999,taz3000,taz3001,taz3002,taz3003,taz3004,taz3005,taz3006,taz3007,taz3008,taz3009,taz3010,taz3011,taz3012,taz3013,taz3014,taz3015,taz3016,taz3017,taz3018,taz3019,taz3020,taz3021,taz3022,taz3023,taz3024,taz3025,taz3026,taz3027,taz3028,taz3029,taz3030
0,AMC_EMP,0.000000,1.000000,3.000000,0.000000,0.000000,111.000000,117.000000,32.000000,61.000000,0.000000,32.000000,1.000000,0.000000,0.000000,0.000000,1.000000,35.000000,0.000000,27.000000,8.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,14.000000,2.000000,0.000000,47.000000,19.000000,14.000000,1.000000,2.000000,7.000000,8.000000,21.000000,0.000000,1.000000,5.000000,0.000000,0.000000,5.000000,23.000000,1.000000,19.000000,0.000000,0.000000,9.000000,18.000000,3.000000,0.000000,0.000000,940.000000,0.000000,9.000000,46.000000,0.000000,1.000000,22.000000,1125.000000,36.000000,737.000000,1450.000000,1.000000,13.000000,1.000000,451.000000,80.000000,11.000000,0.000000,0.000000,6.000000,0.000000,0.000000,1.000000,0.000000,0.000000,5.000000,38.000000,21.000000,40.000000,2.000000,4.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,1.000000,0.000000,137.000000,100.000000,5.000000,1.000000,4.000000,1.000000,9.000000,394.000000,82.000000,1.000000,7.000000,4.000000,0.000000,4.000000,28.000000,306.000000,45.000000,507.000000,48.000000,11.000000,5.000000,16.000000,0.000000,16.000000,43.000000,55.000000,3.000000,5.000000,...,83.000000,591.000000,83.000000,321.000000,30.000000,84.000000,58.000000,27.000000,2.000000,2.000000,11.000000,41.000000,4.000000,35.000000,5.000000,0.000000,0.000000,10.000000,12.000000,9.000000,46.000000,21.000000,67.000000,83.000000,110.000000,5.000000,758.000000,1030.000000,206.000000,18.000000,17.000000,10.000000,103.000000,274.000000,2.000000,80.000000,203.000000,72.000000,5.000000,15.000000,39.000000,247.000000,105.000000,4.000000,1.000000,16.000000,0.000000,11.000000,46.000000,0.000000,8.000000,115.000000,1.000000,65.000000,29.000000,0.000000,172.000000,557.000000,59.000000,14.000000,0.000000,698.000000,138.000000,36.000000,66.000000,8.000000,416.000000,1.000000e+00,13.000000,14.000000,15.000000,35.000000,6.000000,1.000000,104.000000,351.000000,6.000000,8.000000,0.000000,108.000000,2.000000,335.000000,0.000000,5.000000,3.000000,105.000000,7.000000,7.000000,0.000000,27.000000,1.000000,2.000000,187.000000,0.000000,0.000000,42.000000,0.000000,101.000000,0.000000,2.000000,40.000000,11.000000,370.000000,1.000000,166.00

In [ ]:
landuse_alogit.to_csv(os.path.join(output_folder, 'landuse_alogit_master.csv'), index=False)

In [ ]:
def write_alogit_colnames(file, alogit_df):
    written_colnames = []
    with open(file, 'w') as f:
        first_col = alogit_df.columns[0]
        f.write(first_col+'\n')
        f.write('dist\n')
        f.write('lgsum\n')
        for col in alogit_df[1:]:
            if 'taz' in col and '_dist' in col:
                continue
            if 'taz' in col and '_logsum' in col:
                continue
            f.write(col + '\n')
            

## School Location Choice

In [ ]:
# school_location_folder = os.path.join(edb_folder, 'school_location')
# school_co = pd.read_csv(os.path.join(school_location_folder, 'school_location_choosers_combined.csv'))
# school_ca = pd.read_csv(os.path.join(school_location_folder, 'school_location_alternatives_combined.csv'))
# school_landuse = pd.read_csv(os.path.join(school_location_folder, 'school_location_landuse.csv'))

#### remove invalid choosers

In [ ]:
# univ_tazs = school_landuse[school_landuse['COLLEGE'] > 0]['zone_id']
# k_8_tazs = school_landuse[school_landuse['K_8'] > 0]['zone_id']
# G9_12_tazs = school_landuse[school_landuse['G9_12'] > 0]['zone_id']
# k12_tazs = k_8_tazs + G9_12_tazs
# all_school_taz = univ_tazs + k12_tazs
# print(len(univ_tazs), 'TAZs with university enrollment')
# print(len(k12_tazs), 'TAZs with K-12 enrollment')
# print(len(k_8_tazs), 'TAZs with K-8 enrollment')
# print(len(G9_12_tazs), 'TAZs with 9-12 enrollment')

In [ ]:
# remove persons with an invalid choice
# choosers = len(school_co)
# school_co = school_co[((school_co['school_segment'] == 3) & (school_co['override_choice'].isin(univ_tazs)))
#             | ((school_co['school_segment'] == 2) & (school_co['override_choice'].isin(G9_12_tazs)))
#             | ((school_co['school_segment'] == 1) & (school_co['override_choice'].isin(k_8_tazs)))]
# school_ca = school_ca[school_ca['person_id'].isin(school_co['person_id'])]
# valid_choosers = len(school_co)

# print("Number of people removed due to invalid choice: ", choosers - valid_choosers)
# print("Number of valid choosers: ", valid_choosers)

#### prepare land use for ALOGIT

In [ ]:
#cnovert original taz to sequential TAZs
# orig_taz = school_ca.columns.tolist()[2:]
# orig_taz = [int(i) for i in orig_taz]
# orig_taz.sort()
# seq_taz = list(range(1,len(orig_taz)+1))

# taz_xwalk = pd.DataFrame(list(zip(orig_taz, seq_taz)),
#                columns =['survey_choice', 'survey_choice_seq'])

In [ ]:
# # prepare landuse for school location choice estimation in ALOGIT
# schl_landuse_alogit = landuse[landuse['zone_id'].isin(orig_taz)]
# schl_landuse_alogit = schl_landuse_alogit.pivot_table(columns='zone_id')
# schl_landuse_alogit_cols = ['taz' + str(col) for col in schl_landuse_alogit.columns]
# schl_landuse_alogit.columns = schl_landuse_alogit_cols
# schl_landuse_alogit = schl_landuse_alogit.reset_index().rename(columns={'index': 'landuse_var'})

# #write out land use file variable names
# schl_landuse_alogit['landuse_var'].to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'school_location_choice', 
#                                         'school_landuse_variables.csv'), index=False)

# #prepare landuse file in ALOGIT format
# schl_landuse_alogit = schl_landuse_alogit.drop(columns='landuse_var')
# schl_landuse_alogit

In [ ]:
# assert all(is_number(schl_landuse_alogit.dtypes)), "Have Non-Numeric columns!"

In [ ]:
# #write out landuse file for work location choice estimation in ALOGIT
# schl_landuse_alogit.to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'school_location_choice', 
#                                         'school_landuse_alogit.csv'), index=False, header=False)

#### create ALOGIT structured file

In [ ]:
# school_ca_dist = school_ca[school_ca['variable'] == 'util_dist'].drop(columns='variable').set_index('person_id')
# dist_cols = [int(col) for col in school_ca_dist.columns]
# school_ca_dist.columns = dist_cols
# school_ca_dist.sort_index(axis=1, inplace=True)
# dist_cols = ['taz' + str(col) + '_dist' for col in school_ca_dist.columns]
# school_ca_dist.columns = dist_cols
# school_ca_dist

In [ ]:
# school_ca_dist = school_ca[school_ca['variable'] == 'util_dist'].drop(columns='variable').set_index('person_id')
# dist_cols = [int(col) for col in school_ca_dist.columns]
# school_ca_dist.columns = dist_cols
# school_ca_dist.sort_index(axis=1, inplace=True)
# dist_cols = ['taz' + str(col) + '_dist' for col in school_ca_dist.columns]
# school_ca_dist.columns = dist_cols

# school_ca_logsum = school_ca[school_ca['variable'] == 'mode_choice_logsum'].drop(columns='variable').set_index('person_id')
# logsum_cols = [int(col) for col in school_ca_logsum.columns]
# school_ca_logsum.columns = logsum_cols
# school_ca_logsum.sort_index(axis=1, inplace=True)
# logsum_cols = ['taz' + str(col) + '_logsum' for col in school_ca_logsum.columns]
# school_ca_logsum.columns = logsum_cols

# school_ca_alogit = pd.merge(school_ca_dist, school_ca_logsum, on='person_id')
# school_ca_alogit

#### merge person and household characteristics and survey choice

In [ ]:
# school_ca_alogit = pd.merge(school_ca_alogit, asim_per_hh_df, how='left', on='person_id')
# school_co['survey_choice'] = school_co['override_choice']
# school_ca_alogit = pd.merge(school_ca_alogit, school_co[['person_id', 'survey_choice']], how='left', on='person_id')
# # recode survey choice as sequential TAZ
# school_ca_alogit = pd.merge(school_ca_alogit, taz_xwalk[['survey_choice', 'survey_choice_seq']], how='left', on='survey_choice')
# #replace all nulls with 999
# school_ca_alogit.fillna(999, inplace=True)

In [ ]:
# # List of variables to export to ALOGIT
# export_vars = [x for x in school_ca_alogit.columns if (x.endswith('_dist') or x.endswith('_logsum'))]
# export_vars.insert(0,'person_id')
# other_vars = ['pemploy','age','school_segment','income_segment_y','age_0_to_5','age_6_to_12','auto_ownership','survey_choice_seq']
# export_vars = export_vars + other_vars

# #write out alogit input variables
# filename = os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'school_location_choice', 
#                                         'alogit_input_variables.csv')

# with open(filename, 'w', newline='', encoding='utf-8') as f:
#     writer = csv.writer(f)
#     writer.writerow(['variable'])
#     for val in export_vars:
#         writer.writerow([val])


In [ ]:
# #exclude university students
# school_ca_alogit[school_ca_alogit['school_segment']<3][export_vars].to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'school_location_choice', 
#                                         'school_alogit_input.csv'), index=False, header=False)


In [ ]:
# school_ca_alogit[school_ca_alogit['school_segment']<3][export_vars]

In [ ]:
# #TLFD for students
# num_bins = 50
# plt.figure(figsize=(15, 8))
# #bin_seq = list(range(75))
# #bin_seq.append(125)
# n, bins, patches = plt.hist(school_ca_alogit[school_ca_alogit['school_segment']<3]['distance_to_school'], 
#                             bins=list(range(int(max(school_ca_alogit['distance_to_school'])))), facecolor='blue', alpha=0.5)

# plt.title('School TLFD - RTS')
# plt.xlabel('Distance (Miles)')
# plt.ylabel('Frequency')
# plt.savefig(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'school_location_choice', 
#                                         'RTS_TLFD_School.png'))

# plt.show()

## Work Location Choice

In [ ]:
work_location_folder = os.path.join(edb_folder, 'workplace_location')
work_co = pd.read_csv(os.path.join(work_location_folder, 'workplace_location_choosers_combined.csv'))
work_ca = pd.read_csv(os.path.join(work_location_folder, 'workplace_location_alternatives_combined.csv'))
work_landuse = pd.read_csv(os.path.join(work_location_folder, 'workplace_location_landuse.csv'))

In [ ]:
work_co

In [ ]:
dist_check = work_co[work_co['override_choice']> 0].copy().merge(asim_per_hh_df, how='left', on='person_id')
dist_check['model_distance'] = dist_skim[(dist_check['home_zone_id'].astype(int) - 1), (dist_check['model_choice'].astype(int) - 1)]
dist_check['survey_distance'] = dist_skim[(dist_check['home_zone_id'].astype(int) - 1), (dist_check['override_choice'].astype(int) - 1)]


In [ ]:
num_bins = 100
plt.figure(figsize=(15, 8))
#bin_seq = list(range(75))
#bin_seq.append(125)
n, bins, patches = plt.hist(dist_check[dist_check['work_from_home'] == 0]['survey_distance'], 
                            bins=list(range(int(max(dist_check['survey_distance'])))), facecolor='blue', alpha=0.5)
plt.title('Survey TLFD')
plt.xlabel('Distance (Miles)')
plt.ylabel('Frequency')

In [ ]:
work_co

In [ ]:
dist_skim[2665]

In [ ]:
work_ca[work_ca['person_id'] == 14]

In [ ]:
orig_taz = work_ca.columns.tolist()[2:]
orig_taz = [int(i) for i in orig_taz]
seq_taz = list(range(1,len(orig_taz)+1))

taz_xwalk = pd.DataFrame(list(zip(orig_taz, seq_taz)),
               columns =['survey_choice', 'survey_choice_seq'])

In [ ]:
work_ca

In [ ]:
work_ca.columns[2:]

In [ ]:
# Fix the parsing of the state variable

for c in work_ca.columns[2:]:
    work_ca.loc[(work_ca['variable'] == 'util_stay_in_state'), c] = work_ca.loc[(work_ca['variable'] == 'util_stay_in_state'), c].apply(lambda x: 1 if x == 'True' else 0)

work_ca.loc[(work_ca['variable'] == 'util_stay_in_state'), work_ca.columns[2:]]

In [ ]:
asim_per_df

In [ ]:
worker_checks = work_co.merge(asim_per_df[['person_id', 'age', 'ptype']], how = 'left', on = 'person_id')

pd.crosstab(worker_checks.ptype, worker_checks.pemploy)

In [ ]:
worker_checks['age_bin'] = pd.cut(worker_checks.age, [0, 16, 18, 65, np.inf], ['u16', '16-18', '18-65', 'ret'])
worker_checks['age_bin'].value_counts().sort_index()

In [ ]:
# prepare landuse for work location choice estimation in ALOGIT
work_landuse_alogit = landuse[landuse['TOT_EMP']>0]
work_landuse_alogit = work_landuse_alogit.pivot_table(columns='zone_id')
work_landuse_alogit_cols = ['taz' + str(col) for col in work_landuse_alogit.columns]
work_landuse_alogit.columns = work_landuse_alogit_cols
work_landuse_alogit = work_landuse_alogit.reset_index().rename(columns={'index': 'landuse_var'})

#write out land use file variable names
work_landuse_alogit['landuse_var'].to_csv(os.path.join(output_folder, 
                                        'destination_choice', 
                                        'work_location_choice', 
                                        'work_landuse_variables.csv'), index=False)

#prepare landuse file in ALOGIT format
work_landuse_alogit = work_landuse_alogit.drop(columns='landuse_var')
work_landuse_alogit

In [ ]:
landuse

In [ ]:
assert all(is_number(work_landuse_alogit.dtypes)), "Have Non-Numeric columns!"

In [ ]:
#write out landuse file for work location choice estimation in ALOGIT
work_landuse_alogit.to_csv(os.path.join(output_folder, 
                                        'destination_choice', 
                                        'work_location_choice', 
                                        'work_landuse_alogit.csv'), index=False, header=False)

In [ ]:
# remove persons with an invalid choice
workplace_tazs = work_landuse[work_landuse['TOT_EMP'] > 0]['zone_id']  
choosers = len(work_co)
work_co = work_co[work_co['override_choice'].isin(workplace_tazs)]
work_ca = work_ca[work_ca['person_id'].isin(work_co['person_id'])]
valid_choosers = len(work_co)

print("Number of people removed due to invalid choice: ", choosers - valid_choosers)
print("Number of valid choosers: ", valid_choosers)

In [ ]:
work_ca['variable'].value_counts()

In [ ]:
work_ca_dist = work_ca[work_ca['variable'] == 'util_dist'].drop(columns='variable').set_index('person_id')
dist_cols = ['taz' + col + '_dist' for col in work_ca_dist.columns]
work_ca_dist.columns = dist_cols

work_ca_logsum = work_ca[work_ca['variable'] == 'mode_choice_logsum'].drop(columns='variable').set_index('person_id')
logsum_cols = ['taz' + col + '_logsum' for col in work_ca_logsum.columns]
work_ca_logsum.columns = logsum_cols

work_ca_xstate = work_ca[work_ca['variable'] == 'util_stay_in_state'].drop(columns='variable').set_index('person_id').astype(bool).astype(np.uint8)
work_ca_xstate_cols = ['taz' + col + '_statex' for col in work_ca_xstate.columns]
work_ca_xstate.columns = work_ca_xstate_cols

work_ca_alogit = pd.merge(work_ca_dist, work_ca_logsum, on='person_id') #.merge(work_ca_xstate, on = 'person_id')
work_ca_alogit


In [ ]:
work_ca_alogit = pd.merge(work_ca_alogit, asim_per_hh_df, how='left', on='person_id')
work_co['survey_choice'] = work_co['override_choice']
work_ca_alogit = pd.merge(work_ca_alogit, work_co[['person_id', 'survey_choice']], how='left', on='person_id')
# recode survey choice as sequential TAZ
work_ca_alogit = pd.merge(work_ca_alogit, taz_xwalk[['survey_choice', 'survey_choice_seq']], how='left', on='survey_choice')
#replace all nulls with 999
work_ca_alogit.fillna(999, inplace=True)

In [ ]:
for c in work_ca_alogit.columns:
    if c.lower().find("zone") >= 0:
        print(c)

In [ ]:
# The previous version of this used the Asim output of distance_to_work, which was based on the model location, not the survey location
# work_ca_alogit['distance_to_work'] = 
work_ca_alogit.apply(lambda x: dist_skim[x.home_zone_id_x - 1, x.workplace_zone_id - 1], axis = 1)

In [ ]:
work_ca_alogit['work_from_home'].value_counts()

In [ ]:
#distribution by income segments
work_ca_alogit[work_ca_alogit['work_from_home']==0]['income_segment_y'].value_counts().sort_index()

In [ ]:
#TLFD for workers
num_bins = 100
plt.figure(figsize=(15, 8))
#bin_seq = list(range(75))
#bin_seq.append(125)
n, bins, patches = plt.hist(work_ca_alogit[work_ca_alogit['work_from_home']==0]['distance_to_work'], 
                            bins=list(range(int(max(work_ca_alogit['distance_to_work'])))), facecolor='blue', alpha=0.5)
plt.title('WORK TLFD - RTS')
plt.xlabel('Distance (Miles)')
plt.ylabel('Frequency')
plt.savefig(os.path.join(output_folder, 
                                        'destination_choice', 
                                        'work_location_choice', 
                                        'RTS_TLFD.png'))
plt.show()

In [ ]:
print(f"The average trip length is {work_ca_alogit[work_ca_alogit['work_from_home']==0]['distance_to_work'].mean()}")

In [ ]:
print(f"There are {work_ca_alogit[(work_ca_alogit['work_from_home']==0) & (work_ca_alogit['distance_to_work'] == 0)].shape[0]} records with a zero-distance to work")

In [ ]:
print(f"There are {work_ca_alogit[(work_ca_alogit['work_from_home']==0) & (work_ca_alogit['distance_to_work'] < 0)].shape[0]} records with a negative distance to work")

In [ ]:
print(f"There are {work_ca_alogit[(work_ca_alogit['work_from_home']==0) & (work_ca_alogit['distance_to_work'] >= 150)].shape[0]} records with more than 150 miles to work")

In [ ]:
sum(work_ca_alogit['work_from_home']==0)

In [ ]:
sum(work_ca_alogit[work_ca_alogit['work_from_home']==0]['distance_to_work']>=35 )

In [ ]:
work_ca_alogit['pemploy'].value_counts()

In [ ]:
work_ca_alogit.age.hist()

In [ ]:
asim_per_hh_df['income_segment_y'].value_counts().sort_index()

In [ ]:
#Age distribution of workers
num_bins = 20
plt.figure(figsize=(15, 8))
#bin_seq = list(range(75))
#bin_seq.append(125)
n, bins, patches = plt.hist(work_ca_alogit['age'], bins=list(range(int(max(work_ca_alogit['age']))+1)), facecolor='blue', alpha=0.5)
plt.show()

In [ ]:
per_hh_variables = asim_per_hh_df.columns.tolist()
filename = os.path.join(output_folder, 
                                        'destination_choice', 
                                        'work_location_choice', 
                                        'per_hh_variables.csv')

with open(filename, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['variable'])
    for val in per_hh_variables:
        writer.writerow([val])

In [ ]:
work_ca_alogit['home_state_y']

In [ ]:
# List of variables to export to ALOGIT
export_vars = [x for x in work_ca_alogit.columns if (x.endswith('_dist') or x.endswith('_logsum') or x.endswith('_statex'))]
export_vars.insert(0,'person_id')
other_vars = ['pemploy','age','SEX', 'is_student', 'num_children','auto_ownership','income_segment_y','survey_choice_seq', 'home_state_y']
export_vars = export_vars + other_vars

#write out alogit input variables
filename = os.path.join(output_folder, 
                                        'destination_choice', 
                                        'work_location_choice', 
                                        'alogit_input_variables.csv')

with open(filename, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['variable'])
    for val in export_vars:
        writer.writerow([val])

#export_vars[-10:]

In [ ]:
pd.DataFrame(export_vars).to_csv(os.path.join(output_folder, 
                                        'destination_choice', 
                                        'work_location_choice',"export_vars.csv"))

In [ ]:
#exclude workers working from home
work_ca_alogit[work_ca_alogit['work_from_home']==0][export_vars].to_csv(os.path.join(output_folder, 
                                        'destination_choice', 
                                        'work_location_choice', 
                                        'work_alogit_input.csv'), index=False, header=False)

In [ ]:
#write a sample

work_ca_alogit[work_ca_alogit['work_from_home']==0][1:100][export_vars].to_csv(os.path.join(output_folder, 
                                        'destination_choice', 
                                        'work_location_choice', 
                                        'work_alogit_input_subset.csv'), index=False, header=False)


In [ ]:
pd.DataFrame(export_vars).to_csv(os.path.join(output_folder, 
                                        'destination_choice', 
                                        'work_location_choice', 
                                        'work_alogit_colnames.csv'), index=False, header=False)

## Non-Mandatory Tour Destination Choice

In [ ]:
# nm_tour_folder = os.path.join(edb_folder, 'non_mandatory_tour_destination')
# nm_tour_co = pd.read_csv(os.path.join(nm_tour_folder, 'non_mandatory_tour_destination_choosers_combined.csv'))
# nm_tour_ca = pd.read_csv(os.path.join(nm_tour_folder, 'non_mandatory_tour_destination_alternatives_combined.csv'))
# tour_landuse = pd.read_csv(os.path.join(nm_tour_folder, 'non_mandatory_tour_destination_landuse.csv'))

In [ ]:
# nm_jtour_folder = os.path.join(edb_folder, 'joint_tour_destination')
# nm_jtour_co = pd.read_csv(os.path.join(nm_jtour_folder, 'joint_tour_destination_choosers_combined.csv'))
# nm_jtour_ca = pd.read_csv(os.path.join(nm_jtour_folder, 'joint_tour_destination_alternatives_combined.csv'))

In [ ]:
# nm_jtour_ca.head(20)

In [ ]:
# nm_tour_ca.head(20)

In [ ]:
# remove extra column from nm_tour_ca
# nm_tour_ca.drop(columns=['2591'],inplace=True)

In [ ]:
#nm_tour_ca[nm_tour_ca['tour_id']==197]
# nm_tour_ca[nm_tour_ca['tour_id']==197][['tour_id','variable','1','447','448','2445']].head(50)

In [ ]:
# nm_tour_ca.info()

In [ ]:
#convert original taz to sequential TAZs
#join tour DF has the TAZs in the same seq (always check to be sure)
# orig_taz = nm_tour_ca.columns.tolist()[2:]
# orig_taz = [int(i) for i in orig_taz]
# orig_taz.sort()
# seq_taz = list(range(1,len(orig_taz)+1))

# taz_xwalk = pd.DataFrame(list(zip(orig_taz, seq_taz)),
#                columns =['survey_choice', 'survey_choice_seq'])

In [ ]:
# taz_xwalk[taz_xwalk['survey_choice_seq']==1581]

In [ ]:
# prepare landuse for school location choice estimation in ALOGIT
# tour_landuse_alogit = landuse[landuse['zone_id'].isin(orig_taz)]
# tour_landuse_alogit = tour_landuse_alogit.pivot_table(columns='zone_id')
# tour_landuse_alogit_cols = ['taz' + str(col) for col in tour_landuse_alogit.columns]
# tour_landuse_alogit.columns = tour_landuse_alogit_cols
# tour_landuse_alogit = tour_landuse_alogit.reset_index().rename(columns={'index': 'landuse_var'})

# #write out land use file variable names
# tour_landuse_alogit['landuse_var'].to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'tour_landuse_variables.csv'), index=False)

# #prepare landuse file in ALOGIT format
# tour_landuse_alogit = tour_landuse_alogit.drop(columns='landuse_var')
# tour_landuse_alogit

In [ ]:
# assert all(is_number(tour_landuse_alogit.dtypes)), "Have Non-Numeric columns!"

In [ ]:
#write out landuse file for work location choice estimation in ALOGIT
# tour_landuse_alogit.to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'tour_landuse_alogit.csv'), index=False, header=False)

In [ ]:
# individual tours
# nm_tour_ca_dist = nm_tour_ca[nm_tour_ca['variable'] == "@_DIST"].drop(columns='variable').set_index('tour_id')
# dist_cols = [int(col) for col in nm_tour_ca_dist.columns]
# nm_tour_ca_dist.columns = dist_cols
# nm_tour_ca_dist.sort_index(axis=1, inplace=True)
# dist_cols = ['taz' + str(col) + '_dist' for col in nm_tour_ca_dist.columns]
# nm_tour_ca_dist.columns = dist_cols

# nm_tour_ca_logsum = nm_tour_ca[nm_tour_ca['variable'] == 'mode_choice_logsum'].drop(columns='variable').set_index('tour_id')
# logsum_cols = [int(col) for col in nm_tour_ca_logsum.columns]
# nm_tour_ca_logsum.columns = logsum_cols
# nm_tour_ca_logsum.sort_index(axis=1, inplace=True)
# logsum_cols = ['taz' + str(col) + '_logsum' for col in nm_tour_ca_logsum.columns]
# nm_tour_ca_logsum.columns = logsum_cols

# nm_tour_ca_alogit = pd.merge(nm_tour_ca_dist, nm_tour_ca_logsum, on='tour_id')

# # joint tours
# nm_jtour_ca_dist = nm_jtour_ca[nm_jtour_ca['variable'] == "@_DIST"].drop(columns='variable').set_index('tour_id')
# dist_cols = [int(col) for col in nm_jtour_ca_dist.columns]
# nm_jtour_ca_dist.columns = dist_cols
# nm_jtour_ca_dist.sort_index(axis=1, inplace=True)
# dist_cols = ['taz' + str(col) + '_dist' for col in nm_jtour_ca_dist.columns]
# nm_jtour_ca_dist.columns = dist_cols

# nm_jtour_ca_logsum = nm_jtour_ca[nm_jtour_ca['variable'] == 'mode_choice_logsum'].drop(columns='variable').set_index('tour_id')
# logsum_cols = [int(col) for col in nm_jtour_ca_logsum.columns]
# nm_jtour_ca_logsum.columns = logsum_cols
# nm_jtour_ca_logsum.sort_index(axis=1, inplace=True)
# logsum_cols = ['taz' + str(col) + '_logsum' for col in nm_jtour_ca_logsum.columns]
# nm_jtour_ca_logsum.columns = logsum_cols

# nm_jtour_ca_alogit = pd.merge(nm_jtour_ca_dist, nm_jtour_ca_logsum, on='tour_id')

# nm_tour_ca_alogit = nm_tour_ca_alogit.append(nm_jtour_ca_alogit)

# nm_tour_ca_alogit

In [ ]:
# Add choices
######################
# nm_tour_ca_alogit = pd.merge(nm_tour_ca_alogit, asim_tours_per_hh_df, how='left', on='tour_id')
# nm_tour_co['survey_choice'] = nm_tour_co['override_choice']
# nm_jtour_co['survey_choice'] = nm_jtour_co['override_choice']
# nm_tour_co = nm_tour_co[['tour_id', 'survey_choice']].append(nm_jtour_co[['tour_id', 'survey_choice']])
# nm_tour_ca_alogit = pd.merge(nm_tour_ca_alogit, nm_tour_co, how='left', on='tour_id')
# # recode survey choice as sequential TAZ
# nm_tour_ca_alogit = pd.merge(nm_tour_ca_alogit, taz_xwalk[['survey_choice', 'survey_choice_seq']], how='left', on='survey_choice')
# #replace all nulls with 999
# nm_tour_ca_alogit.fillna(999, inplace=True)

In [ ]:
# nm_tour_ca_alogit

In [ ]:
# nm_tour_ca_alogit[nm_tour_ca_alogit['survey_choice']==197]

In [ ]:
#export variable list
# tours_per_hh_variables = asim_tours_per_hh_df.columns.tolist()
# filename = os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'tour_per_hh_variables.csv')

# with open(filename, 'w', newline='', encoding='utf-8') as f:
#     writer = csv.writer(f)
#     writer.writerow(['variable'])
#     for val in tours_per_hh_variables:
#         writer.writerow([val])

In [ ]:
'''
#TLFD for tour destination
num_bins = 50
plt.figure(figsize=(15, 8))
#bin_seq = list(range(75))
#bin_seq.append(125)
n, bins, patches = plt.hist(nm_tour_ca_alogit['tour_type']==4]['distance_to_work'], 
                            bins=list(range(int(max(work_ca_alogit['distance_to_work'])))), facecolor='blue', alpha=0.5)
plt.title('WORK TLFD - RTS')
plt.xlabel('Distance (Miles)')
plt.ylabel('Frequency')
plt.savefig(os.path.join(output_folder, 
                                        'destination_choice', 
                                        'work_location_choice', 
                                        'RTS_TLFD.png'))
plt.show()
'''

In [ ]:
# List of variables to export to ALOGIT
# export_vars = [x for x in nm_tour_ca_alogit.columns if (x.endswith('_dist') or x.endswith('_logsum'))]
# export_vars.insert(0,'tour_id')
# export_vars.remove('destination_logsum')
# export_vars.remove('mode_choice_logsum')
# other_vars = ['pemploy','age','SEX', 'is_student', 'num_children','auto_ownership','income_segment_y',
#               'tour_category','survey_choice_seq']
# export_vars = export_vars + other_vars

# '''
#     'work': 1,
#     'school': 2,
#     'escort': 3,
#     'shopping': 4,
#     'eatout': 5,
#     'othmaint': 6,
#     'social': 7,
#     'othdiscr': 8,
#     'eat': 9,
#     'maint': 10,
# '''

# #write out alogit input variables
# filename = os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'alogit_input_variables.csv')

# with open(filename, 'w', newline='', encoding='utf-8') as f:
#     writer = csv.writer(f)
#     writer.writerow(['variable'])
#     for val in export_vars:
#         writer.writerow([val])


# #write out files for each purpose
# #discretionary
# nm_tour_ca_alogit[nm_tour_ca_alogit['tour_type']==8][export_vars].to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'disc_location_choice', 
#                                         'disc_alogit_input.csv'), index=False, header=False)

# #social/visiting
# nm_tour_ca_alogit[nm_tour_ca_alogit['tour_type']==7][export_vars].to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'visi_location_choice', 
#                                         'visi_alogit_input.csv'), index=False, header=False)

# #eat out
# nm_tour_ca_alogit[nm_tour_ca_alogit['tour_type']==5][export_vars].to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'eati_location_choice', 
#                                         'eati_alogit_input.csv'), index=False, header=False)


# #escorting
# nm_tour_ca_alogit[nm_tour_ca_alogit['tour_type']==3][export_vars].to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'esco_location_choice', 
#                                         'esco_alogit_input.csv'), index=False, header=False)


# #shopping
# nm_tour_ca_alogit[nm_tour_ca_alogit['tour_type']==4][export_vars].to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'shop_location_choice', 
#                                         'shop_alogit_input.csv'), index=False, header=False)

# #maintenance
# nm_tour_ca_alogit[nm_tour_ca_alogit['tour_type']==6][export_vars].to_csv(os.path.join(output_folder, 
#                                         'destination_choice', 
#                                         'main_location_choice', 
#                                         'main_alogit_input.csv'), index=False, header=False)


In [ ]:
# nm_tour_ca_alogit['tour_type'].value_counts()

In [ ]:
# export_vars = [x for x in nm_tour_ca_alogit.columns if (x.endswith('_dist') or x.endswith('_logsum'))]
# export_vars.insert(0,'tour_id')
# export_vars.remove('destination_logsum')
# export_vars.remove('mode_choice_logsum')
# other_vars = ['survey_choice_seq']
# export_vars = export_vars + other_vars

# nm_tour_ca_alogit[nm_tour_ca_alogit['tour_type']==6][export_vars]

In [ ]:
# nm_tour_ca_alogit['survey_choice_seq'].max()

## Atwork Subtours

In [ ]:
# atwork_tour_folder = os.path.join(edb_folder, 'atwork_subtour_destination')
# atwork_tour_co = pd.read_csv(os.path.join(atwork_tour_folder, 'atwork_subtour_destination_choosers_combined.csv'))
# atwork_tour_ca = pd.read_csv(os.path.join(atwork_tour_folder, 'atwork_subtour_destination_alternatives_combined.csv'))

In [ ]:
# atwork_tour_ca.head(15)

In [ ]:
#convert original taz to sequential TAZs
#join tour DF has the TAZs in the same seq (always check to be sure)
# orig_taz = atwork_tour_ca.columns.tolist()[2:]
# orig_taz = [int(i) for i in orig_taz]
# orig_taz.sort()
# seq_taz = list(range(1,len(orig_taz)+1))

# taz_xwalk = pd.DataFrame(list(zip(orig_taz, seq_taz)),
#                columns =['survey_choice', 'survey_choice_seq'])

In [ ]:
taz_xwalk

In [ ]:
# at work subtours
# atwork_tour_ca_dist = atwork_tour_ca[atwork_tour_ca['variable'] == "util_dist"].drop(columns='variable').set_index('tour_id')
# dist_cols = [int(col) for col in atwork_tour_ca_dist.columns]
# atwork_tour_ca_dist.columns = dist_cols
# atwork_tour_ca_dist.sort_index(axis=1, inplace=True)
# dist_cols = ['taz' + str(col) + '_dist' for col in atwork_tour_ca_dist.columns]
# atwork_tour_ca_dist.columns = dist_cols

# atwork_tour_ca_logsum = atwork_tour_ca[atwork_tour_ca['variable'] == 'util_mode_choice_logsum'].drop(columns='variable').set_index('tour_id')
# logsum_cols = [int(col) for col in atwork_tour_ca_logsum.columns]
# atwork_tour_ca_logsum.columns = logsum_cols
# atwork_tour_ca_logsum.sort_index(axis=1, inplace=True)
# logsum_cols = ['taz' + str(col) + '_logsum' for col in atwork_tour_ca_logsum.columns]
# atwork_tour_ca_logsum.columns = logsum_cols

# atwork_tour_ca_alogit = pd.merge(atwork_tour_ca_dist, atwork_tour_ca_logsum, on='tour_id')

In [ ]:
# atwork_tour_ca_alogit

In [ ]:
# Add choices
######################
# atwork_tour_ca_alogit = pd.merge(atwork_tour_ca_alogit, asim_tours_per_hh_df, how='left', on='tour_id')
# atwork_tour_co['survey_choice'] = atwork_tour_co['override_choice']
# atwork_tour_ca_alogit = pd.merge(atwork_tour_ca_alogit, atwork_tour_co, how='left', on='tour_id')
# # recode survey choice as sequential TAZ
# atwork_tour_ca_alogit = pd.merge(atwork_tour_ca_alogit, taz_xwalk[['survey_choice', 'survey_choice_seq']], how='left', on='survey_choice')
# #replace all nulls with 999
# atwork_tour_ca_alogit.fillna(999, inplace=True)

In [ ]:
# # List of variables to export to ALOGIT
# export_vars = [x for x in atwork_tour_ca_alogit.columns if (x.endswith('_dist') or x.endswith('_logsum'))]
# export_vars.insert(0,'tour_id')
# export_vars.remove('destination_logsum')
# export_vars.remove('mode_choice_logsum')
# other_vars = ['pemploy','age','SEX', 'is_student', 'num_children','auto_ownership','income_segment_y',
#               'tour_category','survey_choice_seq']
# export_vars = export_vars + other_vars


# #write out files for each purpose
# #discretionary
# atwork_tour_ca_alogit[export_vars].to_csv(os.path.join(output_folder, 
#                                                        'destination_choice', 
#                                         'atwork_subtour_destination', 
#                                         'atwork_alogit_input.csv'), index=False, header=False)

In [ ]:
# atwork_tour_ca_alogit[export_vars].head()

In [ ]:
# asim_tours_per_hh_df.columns

In [ ]:
# asim_tours_per_hh_df.head()

## Weights by Distance Bin

In [ ]:
# tour_lengths_by_purp_file = r"E:\Projects\Clients\SEMCOG\Tasks\Task9_Estimation\run_dir\data_semcog\tourdistance.csv"
# tour_lengths_by_purp = pd.read_csv(tour_lengths_by_purp_file)
# tour_lengths_by_purp.head()

In [ ]:
# tour_lengths_by_purp['TOURPURP'].value_counts(dropna=False)

In [ ]:
tour_purpose_spa_to_asim_dict = {
    1: 'work',
    2: 'univ',
    3: 'school',
    4: 'escort',
    5: 'shopping',
    6: 'othmaint',
    7: 'eatout',
    8: 'social',
    9: 'othdiscr',
    10: 'othmaint',    # work-related, no counts in this category
    11: 'othdiscr',    # Loop
    12: 'othdiscr',    # Change mode 
}
# tour_lengths_by_purp['tour_purpose'] = tour_lengths_by_purp['TOURPURP'].apply(lambda x: tour_purpose_spa_to_asim_dict[x])
# tour_lengths_by_purp.loc[tour_lengths_by_purp['TOURPURP_RECODE'] == 'At-Work', 'tour_purpose'] = 'atwork'

In [ ]:
# tour_lengths_by_purp['tour_purpose'].value_counts(dropna=False)

In [ ]:
# tour_lengths_by_purp = tour_lengths_by_purp[tour_lengths_by_purp['finalweight'] > 0]

In [ ]:
# tour_lengths_by_purp['unitweight'] = 1

In [ ]:
def calculate_correction_factors(df):
    df = df[df['dist'] > 0]  # don't want to include missing distances in the calculation
    total_weighted = df.finalweight.sum()
    total_unweighted = df.unitweight.sum()
    
    total_weighted_share = 0
    total_unweighted_share = 0
    
    distance_bins = [0, 1, 3, 6, 10, 15]
    for i in range(len(distance_bins)):
        min_dist = distance_bins[i]
        if i == len(distance_bins) - 1:
            max_dist = 9999
        else:
            max_dist = distance_bins[i+1]

            
        bin_df = df[(df['dist'] > min_dist) & (df['dist'] < max_dist)]

        weighted_share = bin_df.finalweight.sum() / total_weighted
        total_weighted_share += weighted_share
        unweighted_share = bin_df.unitweight.sum() / total_unweighted
        total_unweighted_share += unweighted_share
        correction_factor = -1 * np.log(weighted_share / unweighted_share)
        
        print("Dist", min_dist, "-", max_dist, ": weighted share =", round(weighted_share, 3), ", unweighted share =",
              round(unweighted_share, 3)," num entries =", len(bin_df),  ", correction_factor =", round(correction_factor,4))
        
    print("total weighted share =", total_weighted_share, ",  total unweighted share =", total_unweighted_share)
        

In [ ]:
# work_tour_lengths = tour_lengths_by_purp[tour_lengths_by_purp['tour_purpose'] == 'work']
# print("Work correction factors:")
# calculate_correction_factors(work_tour_lengths)

In [ ]:
# school_tour_lengths = tour_lengths_by_purp[tour_lengths_by_purp['tour_purpose'] == 'school']
# print("School correction factors:")
# calculate_correction_factors(school_tour_lengths)

In [ ]:
# escort_tour_lengths = tour_lengths_by_purp[tour_lengths_by_purp['tour_purpose'] == 'escort']
# print("Escort correction factors: ")
# calculate_correction_factors(escort_tour_lengths)

In [ ]:
# shopping_tour_lengths = tour_lengths_by_purp[tour_lengths_by_purp['tour_purpose'] == 'shopping']
# print("Shopping correction factors: ")
# calculate_correction_factors(shopping_tour_lengths)

In [ ]:
# eatout_tour_lengths = tour_lengths_by_purp[tour_lengths_by_purp['tour_purpose'] == 'eatout']
# print("Eatout correction factors: ")
# calculate_correction_factors(eatout_tour_lengths)

In [ ]:
# othmaint_tour_lengths = tour_lengths_by_purp[tour_lengths_by_purp['tour_purpose'] == 'othmaint']
# print("Other Maintenance correction factors: ")
# calculate_correction_factors(othmaint_tour_lengths)

In [ ]:
# othdisc_tour_lengths = tour_lengths_by_purp[tour_lengths_by_purp['tour_purpose'] == 'othdiscr']
# print("Other Discretionary correction factors: ")
# calculate_correction_factors(othdisc_tour_lengths)

In [ ]:
# social_tour_lengths = tour_lengths_by_purp[tour_lengths_by_purp['tour_purpose'] == 'social']
# print("Social correction factors: ")
# calculate_correction_factors(social_tour_lengths)

In [ ]:
# atwork_tour_lengths = tour_lengths_by_purp[tour_lengths_by_purp['tour_purpose'] == 'atwork']
# print("At Work correction factors: ")
# calculate_correction_factors(atwork_tour_lengths)